# Advanced

In [1]:
import numpy as np
from roseau.load_flow import *

In [2]:
ground = Ground()
pref = PotentialRef(ground)  # Set the ground to 0

# Create a MV voltage source
un = 20e3
voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", n=4, ground=ground, voltages=voltages)

# Add a transformer
bus0 = Bus(id="bus0", n=4)
tc = TransformerCharacteristics("160_kVA", "Dyn11", sn=160.0 * 1e3, uhv=un, ulv=400.0, i0=0.023, p0=460.0, psc=2350.0, vsc=0.04)
transformer = DeltaWyeTransformer(id="transfo", bus1=vs, bus2=bus0, transformer_characteristics=tc, tap=1.025)

# Add a LV network
lc = LineCharacteristics.from_name_lv("S_AL_150")
bus1 = Bus(id="bus1", n=4)
bus2 = Bus(id="bus2", n=4)
load_bus1 = Bus(id="load_bus1", n=4)
load_bus2 = Bus(id="load_bus2", n=4)
load_bus3 = Bus(id="load_bus3", n=4)
line1 = ShuntLine(id="line1", n=4, bus1=bus0, bus2=bus1, ground=ground, line_characteristics=lc, length=0.5)
line2 = ShuntLine(id="line2", n=4, bus1=bus1, bus2=bus2, ground=ground, line_characteristics=lc, length=0.4)
line3 = ShuntLine(id="line3", n=4, bus1=bus1, bus2=load_bus1, ground=ground, line_characteristics=lc, length=0.3)
line4 = ShuntLine(id="line4", n=4, bus1=bus2, bus2=load_bus2, ground=ground, line_characteristics=lc, length=0.3)
line5 = ShuntLine(id="line5", n=4, bus1=bus2, bus2=load_bus3, ground=ground, line_characteristics=lc, length=0.4)
load1 = PowerLoad(id="load1", n=4, bus=load_bus1, s=[-1000, -1000, -1000])
load2 = PowerLoad(id="load2", n=4, bus=load_bus1, s=[-1000, -1000, -1000])
load3 = PowerLoad(id="load3", n=4, bus=load_bus1, s=[-100000, 0, 0])

# Create the network
en = ElectricalNetwork.from_element(vs)

In [3]:
en.solve_load_flow(auth=("username", "dev password"))
load_bus3.potentials

Magnitude,[ 366.46183422+230.00065992j -1.02883488-408.71961195j -354.3764284 +203.21914104j -12.86345571 -16.29434761j]
Units,volt


In [4]:
# Let's make the load 3 flexible to eliminate the voltages constraints
en.remove_element("load3")
parameter = FlexibleParameter.p_max_u_production(u_up=240, u_max=250, s_max=10000)
flexible_load = FlexibleLoad(
    id="fl", n=4, bus=load_bus3, s = [-10000, 0, 0], parameters=[parameter, FlexibleParameter.constant(), FlexibleParameter.constant()]
)
en.add_element(flexible_load)

In [5]:
en.solve_load_flow(auth=("username", "dev password"))
load_bus3.potentials

Magnitude,[ 3.55523821e+02+2.05535718e+02j 1.97778107e-01-4.10593456e+02j -3.55643809e+02+2.05058389e+02j -7.51442653e-02+6.04855159e-03j]
Units,volt
